In [3]:
from transformers import AutoTokenizer, AutoModel, AutoConfig
import torch
model_name = "lorenzozan/ME2-BERT"


text = [
     'Faithless is he that says farewell when the road darkens.',
     'The soul is healed by being with children.', 
     'I remembered how we had we had all come to Gatsby’s and guessed at his corruption… while he stood before us concealing an incorruptible dream…',
     'All the variety, all the charm, all the beauty of life is made up of light and shadow, but justice must always remain clear and unbroken.',
     'When tyranny becomes law, rebellion becomes duty.',
     'Reality is merely an illusion, albeit a very persistent one.']  
     
max_length = 256



mf_mapping = {'CH':'CARE/HARM','FC':'FAIRNESS/CHEATING', 'LB':'LOYALTY/BETRAYAL', 'AS':'AUTHORITY/SUBVERSION', 'PD': 'PURITY/DEGRADATION'}


tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)


encoded_input = tokenizer(
    text,
    max_length=max_length,  
    padding="max_length",  
    truncation=True,  
    return_tensors="pt", 
)

input_ids = encoded_input["input_ids"]
attention_mask = encoded_input["attention_mask"]

model.eval()  
with torch.no_grad():
    output = model(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)

for i, tt in enumerate(text):
    print(tt)
    for mf, score in output[i].items():
        print(f'{mf_mapping[mf]} : {score}')
        
    print()




/home/standard/.pyenv/versions/3.10.1/envs/bilaw/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Faithless is he that says farewell when the road darkens.
CARE/HARM : 0.05056
FAIRNESS/CHEATING : 0.01845
LOYALTY/BETRAYAL : 0.8676
AUTHORITY/SUBVERSION : 0.01655
PURITY/DEGRADATION : 0.06524

The soul is healed by being with children.
CARE/HARM : 0.83783
FAIRNESS/CHEATING : 0.02016
LOYALTY/BETRAYAL : 0.42663
AUTHORITY/SUBVERSION : 0.00525
PURITY/DEGRADATION : 0.61056

I remembered how we had we had all come to Gatsby’s and guessed at his corruption… while he stood before us concealing an incorruptible dream…
CARE/HARM : 0.00676
FAIRNESS/CHEATING : 0.04518
LOYALTY/BETRAYAL : 0.02287
AUTHORITY/SUBVERSION : 0.00545
PURITY/DEGRADATION : 0.64035

All the variety, all the charm, all the beauty of life is made up of light and shadow, but justice must always remain clear and unbroken.
CARE/HARM : 0.08769
FAIRNESS/CHEATING : 0.95034
LOYALTY/BETRAYAL : 0.05768
AUTHORITY/SUBVERSION : 0.00725
PURITY/DEGRADATION : 0.06396

When tyranny becomes law, rebellion becomes duty.
CARE/HARM : 0.1599
FAIRNE

You can also run the model without tha LSTM gate and/or the transformation modules.

In [19]:

config = AutoConfig.from_pretrained(model_name, trust_remote_code=True)

config.has_gate = False  
config.has_trans = True  

model = AutoModel.from_pretrained(model_name, config=config, trust_remote_code=True)
text = 'I am proud of my country.'
inputs = tokenizer(
    text,
    max_length=max_length,  
    padding="max_length",  
    truncation=True,  
    return_tensors="pt", 
)

with torch.no_grad():
    print(model(**inputs))

tensor([[0.1263, 0.4236, 0.9987, 0.0916, 0.0888]])
